In [3]:
import os
import io
import pandas as pd
from pandas import DataFrame
import sys
import numpy as np
import json
import pickle
import jsonpickle
import time
import random

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
# gcloud auth application-default login

In [4]:
#df = pd.read_pickle('/home/jm/Documents/testingStuff/my_file.pkl')
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = False)
df = pd.read_sql_table("sawi_tweets_historical", con = engine)

/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_1` CHECK (`user_verified` in (0,1)),'
  util.warn("Unknown schema content: %r" % line)
/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_2` CHECK (`RT` in (0,1))'
  util.warn("Unknown schema content: %r" % line)


In [5]:
df.head()

,index,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,0,2017-09-24 23:54:27,2,None,None,None,0,AmanteTech,Would next #Apple #iOS update cause your exist...,912102964881367040,...,2133,635,632,869739471570325504,"Sydney, New South Wales",AmanteTech,6314,Pacific Time (US & Canada),0,0
1,1,2017-09-24 23:30:33,0,None,None,None,84,supermeg500,RT @MegsSkunkey: iPhone 8 giveaway!!\nhttps://...,912096951532847104,...,61,361,4512,819370348420284417,,Giveaways~,402,None,0,1
2,2,2017-09-24 22:45:06,0,None,None,None,1860,botlamaya,RT @Sudathchamin: iPhone X new price?Ur left k...,912085514282553344,...,10,617,274,941255144,Colombo,Bot Lamaya [beta],51626,New Delhi,0,1
3,3,2017-09-24 22:45:06,0,None,None,None,1860,hikma_5000,RT @Sudathchamin: iPhone X new price?Ur left k...,912085512776798213,...,1388,143,113,791115735200182272,"Mansfield, TX",Hikma🌹🌹🌹,5432,Pacific Time (US & Canada),0,1
4,4,2017-09-24 22:36:56,0,None,None,None,4,Joey2031,RT @BoinkHunter: Beautiful device. \n#iPhoneX ...,912083456552783873,...,853,66,75,705859071044423680,,Joey,646,None,0,1


In [6]:
df.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,33775.000000,33775.000000,33775.000000,3.377500e+04,33775.000000,3.377500e+04,33775.000000,3.377500e+04,3.377500e+04,33775.000000,33775.000000
mean,49.105522,0.863094,4347.657617,9.146381e+17,8463.428483,9.056939e+03,1863.045063,1.829745e+17,4.661424e+04,0.013235,0.522309
std,28.846371,44.781205,14117.690974,7.428145e+14,23336.579759,1.089850e+05,11501.285621,3.435277e+17,1.322553e+05,0.114280,0.499509
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,7.300000e+02,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.142344e+17,114.000000,1.010000e+02,133.000000,2.891535e+08,1.491500e+03,0.000000,0.000000
50%,49.000000,0.000000,2.000000,9.146147e+17,1302.000000,3.800000e+02,358.000000,1.664059e+09,8.455000e+03,0.000000,1.000000
75%,74.000000,0.000000,946.000000,9.153387e+17,7142.500000,1.322000e+03,977.000000,4.191447e+09,3.554250e+04,0.000000,1.000000
max,99.000000,8016.000000,142954.000000,9.153658e+17,595418.000000,9.360144e+06,851643.000000,9.153586e+17,2.210810e+06,1.000000,1.000000


In [7]:
df.count()

index                      33775
created_at                 33775
favorite_count             33775
in_reply_to_screen_name     2854
in_reply_to_status_id       2392
in_reply_to_user_id         2854
retweet_count              33775
screen_name                33775
text                       33775
tweet_id                   33775
tweet_location                67
user_created_at            33775
user_description           33775
user_favorites_count       33775
user_followers_count       33775
user_friends_count         33775
user_id                    33775
user_location              33775
user_name                  33775
user_statuses_count        33775
user_time_zone             20297
user_verified              33775
RT                         33775
dtype: int64

In [8]:
dfs = df.drop_duplicates(subset = 'tweet_id', inplace=False)

In [9]:
dfs.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,33147.000000,33147.000000,33147.000000,3.314700e+04,33147.000000,3.314700e+04,33147.000000,3.314700e+04,3.314700e+04,33147.000000,33147.000000
mean,49.159260,0.873654,4429.207198,9.146779e+17,8578.360515,9.173978e+03,1868.411138,1.790865e+17,4.703215e+04,0.013244,0.522008
std,28.845084,45.203064,14238.239951,6.823812e+14,23506.335610,1.099985e+05,11584.650198,3.404452e+17,1.324933e+05,0.114320,0.499523
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,7.300000e+02,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.142362e+17,119.500000,1.050000e+02,135.000000,2.897296e+08,1.544000e+03,0.000000,0.000000
50%,49.000000,0.000000,2.000000,9.146165e+17,1343.000000,3.840000e+02,361.000000,1.643214e+09,8.720000e+03,0.000000,1.000000
75%,74.000000,0.000000,1168.000000,9.153392e+17,7289.000000,1.322000e+03,978.000000,4.056864e+09,3.601600e+04,0.000000,1.000000
max,99.000000,8016.000000,142954.000000,9.153658e+17,595418.000000,9.360144e+06,851643.000000,9.153586e+17,2.210810e+06,1.000000,1.000000


In [10]:
dfs.count()

index                      33147
created_at                 33147
favorite_count             33147
in_reply_to_screen_name     2853
in_reply_to_status_id       2392
in_reply_to_user_id         2853
retweet_count              33147
screen_name                33147
text                       33147
tweet_id                   33147
tweet_location                62
user_created_at            33147
user_description           33147
user_favorites_count       33147
user_followers_count       33147
user_friends_count         33147
user_id                    33147
user_location              33147
user_name                  33147
user_statuses_count        33147
user_time_zone             19987
user_verified              33147
RT                         33147
dtype: int64

In [11]:
dfs.to_csv("twitter_dataset_v2.csv", float_format='{:f}'.format)